In [ ]:
from programs import CutadaptCommando
%cd ..

In [ ]:
cut = CutadaptCommando('test_yaml.yaml')

In [ ]:
for a,b in cut.create_command():
    print(a)

In [4]:
from programs import FeatureCountsCommando
%cd ..

/Users/williamrosenbaum/Bioinformatics/gentools


In [5]:
fet = FeatureCountsCommando('test_yaml.yaml')

In [7]:
fet.input_files

[]